In [28]:
import numpy as np
import matplotlib.pyplot as plt
Pt = np.loadtxt("Pt/DOS.dat")
Fe = np.loadtxt("Fe/DOS.dat")

In [46]:
def truncate_data(E_limits:tuple, data):
    E_min, E_max = E_limits
    first = True
    for row in data:
        E = row[0]
        if E > E_min and E < E_max:
            if first:
                arr = row.copy().reshape(1,4)
                first = False
            else:
                arr = np.append(arr,row.reshape((1,4)), axis = 0)
    return arr
    

def interpolate(E, data):
    for row1 in data:
        if row1[0] > E:
            offset = E - row0[0]
            delta_E = row1[0]-row0[0]
            delta_spin_up = row1[1] - row0[1]
            delta_spin_down = row1[2] - row0[2]
            up_slope = delta_spin_up/delta_E
            down_slope = delta_spin_down/delta_E
            new_up = row0[1] + offset * up_slope
            new_down = row0[2] + offset * down_slope
            new_tot = new_up + new_down
            new_row = np.array([E, new_up, new_down, new_tot])
            return new_row
        row0 = row1
        
def trapz(f, x):
    vals = []
    for a,b,x0,x1 in zip(f[:-1], f[1:], x[:-1], x[1:]):
        dx = x1 - x0
        val = (a+b)*dx/2
        vals.append(val)
    vals_arr = np.array(vals)
    return vals_arr        

def integrate_DOS(E_DOS_intervals, data):
    E_0 = interpolate(E_DOS_intervals[0], data)
    DOS_pdf = E_0.reshape(1,4)
    for E in E_DOS_intervals[1:]:
        row = interpolate(E, data)
        DOS_pdf = np.append(DOS_pdf, row.reshape((1,4)), axis = 0)
    spin_up = DOS_pdf[:,1]
    spin_down = DOS_pdf[:,2]
    density_up = trapz(spin_up, E_DOS_intervals)
    density_down = trapz(spin_down, E_DOS_intervals)
    density_up_down = np.append(density_up.reshape((-1,1)),density_down.reshape((-1,1)), axis = 1 )
    return density_up_down

In [53]:
E_trunc = (-0.5, 2)
Fe_data = truncate_data(E_trunc, Fe)
Pt_data = truncate_data(E_trunc, Pt)

step = 0.125
start = 0
end = 1.5
DOS_eV = np.arange(start, end + step, step)

Fe_DOS_unitcell = integrate_DOS(DOS_eV, Fe_data)
Pt_DOS_unitcell = integrate_DOS(DOS_eV, Pt_data)

Fe_conversion = 1000/23.55
Pt_conversion = 1000/60.43

Fe_DOS = Fe_DOS_unitcell * Fe_conversion
Pt_DOS = Pt_DOS_unitcell * Pt_conversion

np.savetxt("Fe_DOS.txt", Fe_DOS, fmt='%1.3f')
np.savetxt("Pt_DOS.txt", Pt_DOS, fmt='%1.3f')


[[-0.412414  1.661276  2.045079  3.706355]]
[[-0.491104  2.599687  2.599687  5.199374]]
[[ 6.70383574  3.12578656]
 [ 4.98018386  2.61699487]
 [ 3.29417695  2.51364384]
 [ 2.22468432  2.68629671]
 [ 1.78379818  3.16394365]
 [ 1.6645136   4.08946945]
 [ 1.62958263  5.51784672]
 [ 1.60462629  7.251109  ]
 [ 1.58305274  9.05920183]
 [ 1.56048949 11.00665031]
 [ 1.53690608 13.11227358]
 [ 1.51620186 14.79721532]]
[[6.93163065 6.93163065]
 [5.61694618 5.61694618]
 [4.16405676 4.16405676]
 [2.87990747 2.87990747]
 [1.90117695 1.90117695]
 [1.35383654 1.35383654]
 [1.16507588 1.16507588]
 [1.12167951 1.12167951]
 [1.11219002 1.11219002]
 [1.10714219 1.10714219]
 [1.10248115 1.10248115]
 [1.09803096 1.09803096]]
